In [61]:
import pandas as pd
import requests
import lxml
import html5lib
import hvplot.pandas
import matplotlib.pyplot as plt
#from bs4 import beautifulsoup4
#https://developers.tiktok.com/doc/research-api-specs-query-videos/
#https://data.cityofnewyork.us/Public-Safety/NYPD-Arrest-Data-Year-to-Date-/uip8-fykc
# Library for opening url and creating 
# requests
import urllib.request

# pretty-print python data structures
from pprint import pprint

# for parsing all the tables present 
# on the website
from html_table_parser.parser import HTMLTableParser

In [62]:
#csv_crime_data = pd.read_csv("Police_Department_Incident_Reports__2018_to_Present.csv")
clean_crime = pd.read_csv("police_reports_clean.csv")

In [63]:
#df_crime = pd.DataFrame(csv_crime_data)
df = pd.DataFrame(clean_crime)

In [64]:
df_crime_cleaned = df_crime[["Incident Date", "Latitude", "Longitude", "Filed Online", "Analysis Neighborhood", "Incident Description", "Incident Category", "Incident Subcategory"]]
df_crime_cleaned["Filed Online"] = df_crime_cleaned["Filed Online"].fillna("False")
#df_crime_cleaned

C:\Users\vidul\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [65]:
df_crime_cleaned[['Year', 'Month', 'Day']] = df_crime_cleaned["Incident Date"].str.split("/",expand=True)

df_crime_cleaned['Year'] = df_crime_cleaned['Year'].astype('int')
df_crime_cleaned['Month'] = df_crime_cleaned['Month'].astype('int')
del df_crime_cleaned["Day"]

df_crime_cleaned["Year/Month"] = df_crime_cleaned['Year'] + (df_crime_cleaned['Month'] / 12) #Months as decimals 

#df_crime_cleaned #Check analysis neighborhoods for neighborhood name 

C:\Users\vidul\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\vidul\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\vidul\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [40]:
df_crime_online_true = df_crime_cleaned.loc[df_crime_cleaned["Filed Online"] == True, :]
df_crime_online_true_dropna = df_crime_online_true.dropna()

df_crime_online_false = df_crime_cleaned.loc[df_crime_cleaned["Filed Online"] == "False", :]
df_crime_online_false_dropna = df_crime_online_false.dropna()

In [41]:
df_crime_online_false_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 552168 entries, 0 to 706064
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Incident Date          552168 non-null  object 
 1   Latitude               552168 non-null  float64
 2   Longitude              552168 non-null  float64
 3   Filed Online           552168 non-null  object 
 4   Analysis Neighborhood  552168 non-null  object 
 5   Incident Description   552168 non-null  object 
 6   Incident Category      552168 non-null  object 
 7   Incident Subcategory   552168 non-null  object 
 8   Year                   552168 non-null  int32  
 9   Month                  552168 non-null  int32  
 10  Year/Month             552168 non-null  float64
dtypes: float64(3), int32(2), object(6)
memory usage: 46.3+ MB


In [19]:
#df_crime_online_false_dropna.groupby("Analysis Neighborhood").count().sort_values("Latitude", ascending=False)

In [42]:
# nonviolent crimes (Change Weapons Carrying Etc -> Weapons Carrying)
# misc (Change Suspicious Occ -> Suspicious Activity)

violent_crimes = ["Assult", "Larceny Theft", "Burglary", "Motor Vehicle Theft", "Robbery", 
                  "Weapons Offense", "Arson", "Rape", "Homicide", 
                  "Human Trafficing (A) Commercial Sex Acts"]
nonviolent_crimes = ["Malicious Mischeif", "Other Miscellaneous", "Fraud", "Drug Offense", 
                     "Disorderly Conduct", "Offences Against The Family And Children", 
                     "Traffic Violation Arrest", "Weapons Carrying Etc", 
                     "Stolen Property", "Forgery and Counterfeiting", "Traffic Collision", 
                     "Vandalism", "Prostitution", "Embezzlement", "Civil Sidewalks", 
                     "Sex Offense", "Drug Violation"]
misc_crimes = ["Lost Property", "Missing Person", "Suspicious Occ", "Miscellaneous Investigation", 
        "Other Offenses", "Courtesy Report", "Fire Report", "Vehicle Impounded", "Suicide"]
get_rid_of = ["Non-Criminal", "Recovered Vehicles", "Lost Property", "Warrant", "Other", 
              "Other Offences", "Case Closure", "Vehicle Misplaced", "Suspicious"] 

In [71]:
#df_crime_online_false_dropna[df_crime_online_false_dropna["Incident Category"] == violent_crimes, :]
#df.loc[df['col1'].isin([value1, value2, value3, ...])]
df_violent_crimes = df_crime_online_false_dropna.loc[df_crime_online_false_dropna["Incident Category"].isin(violent_crimes)]
df_violent_crimes = df_violent_crimes.reset_index(drop=True) # - clean up columns or separate date for plotting over time 
df_violent_crimes #.to_csv("SF_ViolentCrimes.csv")

,Incident Date,Latitude,Longitude,Filed Online,Analysis Neighborhood,Incident Description,Incident Category,Incident Subcategory,Year,Month,Year/Month
0,2021/11/23,37.773039,-122.430629,False,Hayes Valley,Arson,Arson,Arson,2021,11,2021.916667
1,2021/06/17,37.712349,-122.418283,False,Visitacion Valley,Arson of Vehicle,Arson,Arson,2021,6,2021.500000
2,2021/10/12,37.797750,-122.415384,False,Nob Hill,"Theft, Vehicle Strip, >$950",Larceny Theft,Larceny - Auto Parts,2021,10,2021.833333
3,2021/06/18,37.794021,-122.404711,False,Chinatown,Arson,Arson,Arson,2021,6,2021.500000
4,2021/06/17,37.807979,-122.417716,False,North Beach,"License Plate, Stolen",Larceny Theft,Theft From Vehicle,2021,6,2021.500000
...,...,...,...,...,...,...,...,...,...,...,...
199495,2023/03/13,37.807483,-122.413975,False,North Beach,"Theft, From Locked Vehicle, >$950",Larceny Theft,Larceny - From Vehicle,2023,3,2023.250000
199496,2023/03/12,37.726950,-122.476039,False,Lakeshore,"Burglary, Store, Forcible Entry",Burglary,Burglary - Commercial,2023,3,2023.250000
199497,2023/03/13,37.780656,-122.472312,False,Outer Richmond,"Weapon, Deadly, Imitation or Laser Scope, Exhi...",Weapons Offense,Weapons Offense,2023,3,2023.250000
199498,2023/03/12,37.770709,-122.430167,False,Hayes Valley,"Vehicle, Stolen, Auto",Motor Vehicle Theft,Motor Vehicle Theft,2023,3,2023.250000


In [73]:
df_nonviolent_crimes = df_crime_online_false_dropna.loc[df_crime_online_false_dropna["Incident Category"].isin(nonviolent_crimes)]
df_nonviolent_crimes = df_nonviolent_crimes.reset_index(drop=True)
df_nonviolent_crimes.to_csv("SF_NonviolentCrimes.csv")

In [48]:
df_misc_crimes = df_crime_online_false_dropna.loc[df_crime_online_false_dropna["Incident Category"].isin(misc_crimes)]
df_misc_crimes.to_csv("SF_MscCrimes.csv")

In [16]:
#df_crime_online_false_dropna.to_csv("SF_Crime_2018_2023_plot.csv")

In [59]:
df.groupby("Analysis Neighborhood").count()

,Incident Datetime,Incident Date,Incident Year,Incident Day of Week,Report Datetime,Report Type Description,Filed Online,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,Police District,Latitude,Longitude,Incident Time,Report Time,Report Date
Analysis Neighborhood,,,,,,,,,,,,,,,,,,
Bayview Hunters Point,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807,28807
Bernal Heights,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960,8960
Castro/Upper Market,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402,12402
Chinatown,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169,9169
Excelsior,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554,8554
Financial District/South Beach,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378,36378
Glen Park,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647,2647
Golden Gate Park,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382,4382
Haight Ashbury,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409,6409
